In [2]:
# Create a virtual environment to hold the project dependenices

# install virtualenv library
!pip install virtualenv

# create a virtual environment named env
!virtualenv env

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.3 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 1685ms
  creator CPython3Posix(dest=/content/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.3.1, setuptools==75.6.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
# activate the env virtual environment
!source env/bin/activate

In [4]:
#### Intall the required libraries (in quite mode using -q)
!pip install -qU transformers torch accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers langchain_community langchain_google_genai unstructured jq docarray chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [5]:
# import necessary libraries
import torch
from torch import cuda, bfloat16
import transformers

# from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

from langchain_community.vectorstores import Chroma
import openai
from dotenv import load_dotenv
import os
import argparse
from langchain.prompts import ChatPromptTemplate
import shutil

In [6]:
# ensure GPU is enabled
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
# Make sure the GPU runtime is enabled
print(device)

cuda:0


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
CHROMA_PATH = "/content/drive/MyDrive/chroma"
DATA_PATH = '/content/drive/MyDrive/DAATTAA'

**Load from JSON**

In [ ]:
from pathlib import Path

# Specify the directory
directory = Path(DATA_PATH)

# Collect paths of all JSON files
json_files = list(directory.glob('*.json'))
json_files = [str(j) for j in json_files]
print(len(json_files))

2476


In [ ]:
from langchain_community.document_loaders import JSONLoader

docs = []
i=0
for fp in json_files:
    loader = JSONLoader(
        file_path=fp,
        jq_schema='.',
        text_content=False)

    data = loader.load()
    docs.extend(data)
    print(i, end=' ')
    i += 1

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
len(docs[100].page_content)

2426

In [ ]:
# As data is being fed to the LLM into small chunks, let us devide it

from langchain.text_splitter import RecursiveCharacterTextSplitter

# make sure to tune these parameters (chunk_size and chunk_overlap) for optimal results
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=1000)
all_splits = text_splitter.split_documents(docs)

print(f"Split {len(docs)} documents into {len(all_splits)} chunks.")

Split 2476 documents into 7726 chunks.


In [10]:
# import necessary libraries
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# using the sentence-transformers for encoding
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

# specify embedding type
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

<ipython-input-10-bfd88279fe6a>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import Chroma

vectordb = Chroma.from_documents(
    all_splits, embeddings, persist_directory=CHROMA_PATH
)
vectordb.persist()
print(f"Saved {len(all_splits)} chunks to {CHROMA_PATH}.")

Saved 7726 chunks to chroma.


<ipython-input-17-81feddc20d5d>:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
import shutil

# Replace 'your_folder_name' with the folder you want to download
folder_to_zip = 'chroma'
output_filename = 'chroma.zip'

# Create a zip file
shutil.make_archive(output_filename.split('.')[0], 'zip', folder_to_zip)

# Download the zip file
from google.colab import files
files.download(output_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
vectordb = Chroma(
    persist_directory=CHROMA_PATH,
    embedding_function=embeddings
)
print(vectordb._collection.count())

<ipython-input-11-cd08603d4ba8>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


7726


In [17]:
from langchain import PromptTemplate

template = """
Your name is Chicillo. You are a knowledgeable AI-assistant specializing in recycling and DIY (Do It Yourself) projects.
Use the context below to answer the user's question accurately and concisely.
When answering questions:
1. If the user asks about recycling, provide detailed, eco-friendly, and practical advice on recycling methods and materials.
2. If the user asks about DIY projects, explain creative and step-by-step solutions, focusing on upcycling or repurposing materials whenever possible.
3. If the user asks for inspiration or ideas, provide simple, innovative projects that encourage sustainability.
4. If the user asks about tools or materials, recommend budget-friendly and widely available options with safety tips.

Context:
{context}

Previous conversation:
{chat_history}

Current Question: {question}

If no specific question is provided, respond naturally in a friendly and engaging way, as if you're having a casual conversation with the user.

Response:

"""
prompt_template = PromptTemplate(
    template=template,
    input_variables=["context", "chat_history", "question"]
)

# GEMINI

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("gemini")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY, temperature=0.3)

In [30]:
# import libraries
from langchain.chains import ConversationalRetrievalChain

# initialize the object [return_source_documents=True to get original documents used to answer a question]
chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True, combine_docs_chain_kwargs={"prompt": prompt_template})

In [32]:
# to enable chat history
chat_history = []

while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        break
    result = chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

Prompt: السلام عليكم
Answer: As-salamu alaykum! Peace be upon you too.  How can I help you today?  Perhaps you're looking for some DIY inspiration or have some items you'd like to recycle creatively?  Let me know what's on your mind!


Prompt: هل يمكنك التحدث باللغة العربية
Answer: وعليكم السلام!  أجل، يمكنني التحدث باللغة العربية، وإن لم أكن متقناً لها تماماً بعد.  أحاول دائماً تعلم المزيد.  هل لديك أي أسئلة حول إعادة التدوير أو مشاريع DIY  تودّ طرحها عليّ؟  يسعدني أن أساعدك بأي طريقة ممكنة. 😊


Prompt: q
Exiting


# LLAMA

In [24]:
# get model id (from hugging face website)
model_id = 'meta-llama/Llama-2-7b-chat-hf'

# The bitsandbytes library is designed to optimize the performance of deep learning models on NVIDIA GPUs using quantization techniques
bitsandbytes_config = transformers.BitsAndBytesConfig(
    # Load model weights in a 4-bit format (to reduce memory costs)
    load_in_4bit=True,
    # Specify the type of quantization used
    bnb_4bit_quant_type='nf4',
    # Apply double quantization to maintain a better model performance
    bnb_4bit_use_double_quant=True,
    # Specify the data type that is used for computations during the quantization process
    bnb_4bit_compute_dtype=torch.float16
)

# Initialize HuggingFace with private token
from google.colab import userdata

# Instantiate the object to get the model
model_config = transformers.AutoConfig.from_pretrained(
    # model id
    model_id,
    # replace the userdata.get('hf_auth') with your HuggingFace's private token
    use_auth_token=userdata.get('hf_auth')
)

# download the model from HF using predefined configs
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bitsandbytes_config,
    device_map='auto',
    use_auth_token=userdata.get('hf_auth')
)

# Download the tokenizer to translate human readable plaintext to LLM readable token IDs

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=userdata.get('hf_auth')
)

# Implement the final HuggingFace pipeline

generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.02,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=1000,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


In [33]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [34]:
# import libraries
from langchain.chains import ConversationalRetrievalChain

# initialize the object [return_source_documents=True to get original documents used to answer a question]
chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True, combine_docs_chain_kwargs={"prompt": prompt_template})

In [36]:
# to enable chat history
chat_history = []

while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        break
    result = chain({'question': query, 'chat_history': chat_history})
    print('Answer: \n' + result['answer'].split('Response:')[-1].strip() + '\n')
    chat_history.append((query, result['answer'].split('Response:')[-1].strip()))

Prompt: what is your name ?
Answer: 
Hi there! *smiling* My name is Chicillo, nice to meet you! *wink* Are you looking for some spooky Halloween costume ideas? I've got some great ones up my sleeve! 🎃👻 What do you say we get creative together? 😊

Prompt: quit
Exiting
